In [1160]:
from src.kr6 import KR6
import modern_robotics as mr
import numpy as np
from tpk4170.visualization import Viewer
from tpk4170.models import Grid, Axes
from src.dh import DHFrame
import src.tools as tools
import sympy as sp

In [1161]:
r1,r2,r3,r4,a1,a2 = 0.4, 0.455, 0.420, 0.08, 0.025, 0.035
r3_o = np.sqrt(r3**2 + a2**2) # the distance between joint 3 and the wrist
Slist = np.array([[0,0,-1,0,0,0],
                  [0,1,0,-r1,0,a1],
                  [0,1,0,-r1,0,(r2+a1)],
                  [-1,0,0,0,-(r1+a2),0],
                  [0,1,0,-(r1+a2),0,(r2+r3+a1)],
                  [-1,0,0,0,-(r1+a2),0]]).T

M = np.array([[0,0,1,r2+r3+r4+a1],
              [0,1,0,0],
              [-1,0,0,r1+a2],
              [0,0,0,1]])

In [1162]:

thetalist = np.array([33,-63,-34,15,33,33])*np.pi/180
thetalist = np.array([-90,-90  ,90,30,30,30])*np.pi/180 #Write the thetas to make a 
                                                        #transformationmatrix T_SB to
                                                        #test the analytical solver

T = mr.FKinSpace(M,Slist,thetalist)

print(T)

#Get position of the wrist


T_SB = T

#Translate -80mm along z-axis of endframe

T_BW = np.array([[1,0,0,0],
                 [0,1,0,0],
                 [0,0,1,-0.08],
                 [0,0,0,1]])

T_SW = np.dot(T_SB,T_BW)
print(T_SW.round(2))


pw = np.array([T_SW[0][3],T_SW[1][3],T_SW[2][3]])
theta11 = -np.arctan2(pw[1],pw[0])
theta12 = -np.arctan2(-pw[1],-pw[0])




[[ 0.8080127  -0.53349365  0.25        0.02      ]
 [-0.4330127  -0.25        0.8660254   0.51428203]
 [-0.39951905 -0.8080127  -0.4330127   0.85535898]
 [ 0.          0.          0.          1.        ]]
[[ 0.81 -0.53  0.25  0.  ]
 [-0.43 -0.25  0.87  0.45]
 [-0.4  -0.81 -0.43  0.89]
 [ 0.    0.    0.    1.  ]]


In [1163]:
#Forward

Px, Py, Pz =pw[0] - np.cos(theta11) * a1, \
            pw[1] + np.sin(theta11) * a1, \
            pw[2] - r1


c3 = (Px**2 + Py**2 + Pz**2 - r2**2 - r3_o**2) / (2 * r2 * r3_o)

s3p = np.sqrt(1-c3**2)
s3n = -s3p

theta31 = np.arctan2(s3p,c3) + np.arctan2(a2,r3)
theta32 = np.arctan2(s3n,c3) + np.arctan2(a2,r3)

c2pp = (r2 + r3_o * c3) * np.sqrt(Px**2 + Py**2) + r3_o * s3p * Pz
c2np = -(r2 + r3_o * c3) * np.sqrt(Px**2 + Py**2) + r3_o * s3p * Pz
c2pn = (r2 + r3_o * c3) * np.sqrt(Px**2 + Py**2) + r3_o * s3n * Pz
c2nn = -(r2 + r3_o * c3) * np.sqrt(Px**2 + Py**2) + r3_o * s3n * Pz

s2np = (r2 + r3_o*c3) * Pz - r3_o * s3p * np.sqrt(Px**2 + Py**2)
s2pp = (r2 + r3_o*c3) * Pz + r3_o * s3p * np.sqrt(Px**2 + Py**2)
s2nn = (r2 + r3_o*c3) * Pz - r3_o * s3n * np.sqrt(Px**2 + Py**2)
s2pn = (r2 + r3_o*c3) * Pz + r3_o * s3n * np.sqrt(Px**2 + Py**2)

theta21 = np.arctan2(s2np,c2pp) 
# theta22 = np.arctan2(s2pp,c2np)
theta23 = np.arctan2(s2nn,c2pn) 
# theta24 = np.arctan2(s2pn,c2nn) 

In [1164]:
#Backward
Px, Py, Pz =pw[0] - np.cos(theta12) * a1, \
            pw[1] + np.sin(theta12) * a1, \
            pw[2] - r1

c3 = (Px**2 + Py**2 + Pz**2 - r2**2 - r3_o**2) / (2 * r2 * r3_o)

s3p = np.sqrt(1-c3**2)
s3n = -s3p

theta33 = np.arctan2(s3p,c3) + np.arctan2(a2,r3)
theta34 = np.arctan2(s3n,c3) + np.arctan2(a2,r3)

c2pp = (r2 + r3_o * c3) * np.sqrt(Px**2 + Py**2) + r3_o * s3p * Pz
c2np = -(r2 + r3_o * c3) * np.sqrt(Px**2 + Py**2) + r3_o * s3p * Pz
c2pn = (r2 + r3_o * c3) * np.sqrt(Px**2 + Py**2) + r3_o * s3n * Pz
c2nn = -(r2 + r3_o * c3) * np.sqrt(Px**2 + Py**2) + r3_o * s3n * Pz

s2np = (r2 + r3_o*c3) * Pz - r3_o * s3p * np.sqrt(Px**2 + Py**2)
s2pp = (r2 + r3_o*c3) * Pz + r3_o * s3p * np.sqrt(Px**2 + Py**2)
s2nn = (r2 + r3_o*c3) * Pz - r3_o * s3n * np.sqrt(Px**2 + Py**2)
s2pn = (r2 + r3_o*c3) * Pz + r3_o * s3n * np.sqrt(Px**2 + Py**2)

# theta21 = np.arctan2(s2np,c2pp) 
theta22 = np.arctan2(s2pp,c2np)
# theta23 = np.arctan2(s2nn,c2pn) 
theta24 = np.arctan2(s2pn,c2nn) 




config = np.array([[theta11, -theta23, theta31], # forward over elbow
                  [theta11,-theta21, theta32], # forward under elbow
                  [theta12, -theta24, theta33], # backward under elbow
                  [theta12, -theta22, theta34]]) # backward over elbow
config*180/np.pi

array([[ -90.        ,  -90.        ,   90.        ],
       [ -90.        ,   -8.79741071,  -80.47271662],
       [  90.        , -171.27348812,   83.28079896],
       [  90.        ,  -96.33953303,  -73.75351558]])

Thinking of the wrist as a -ZY-Z Euler config

In [1165]:
theta = config[2]
t1,t2,t3 = theta[0],theta[1],theta[2]
R_S3 = tools.rotz(-t1) @ tools.roty(t1) @ tools.roty(t2)

R_S3

R_SB = T_SB[:3,:3]

R_SB

R_3B = np.linalg.inv(R_S3) @ R_SB

R_SB

array([[ 0.8080127 , -0.53349365,  0.25      ],
       [-0.4330127 , -0.25      ,  0.8660254 ],
       [-0.39951905, -0.8080127 , -0.4330127 ]])

In [1166]:
theta61 = np.arctan2(-R_3B[2,1],-R_3B[2,0])
theta62 = np.arctan2(R_3B[2,1],R_3B[2,0])

theta62 * 180/np.pi


-142.88806399347678

In [1167]:
theta41 = np.arctan2(-R_3B[1,2],R_3B[0,2])
theta42 = np.arctan2(R_3B[1,2],-R_3B[0,2])

theta42*180/np.pi

24.08053132624663

In [1168]:
theta51 = np.arctan2(np.sqrt(R_3B[2,1]**2 + R_3B[2,0]**2), R_3B[2,2])
theta51 = np.arctan2(np.sqrt(1-R_3B[2,2]**2), R_3B[2,2])

theta52 = np.arctan2(-np.sqrt(R_3B[2,1]**2 + R_3B[2,0]**2), R_3B[2,2])
theta52 = np.arctan2(-np.sqrt(1-R_3B[2,2]**2), R_3B[2,2])

theta52*180/np.pi

-37.786051075927915

In [1169]:
# for i in config[:1,:]:
for i in config:
    theta = i
    t1,t2,t3 = theta[0],theta[1],theta[2]
    R_S3 = tools.rotz(-t1) @ tools.roty(t2) @ tools.roty(t3)

    R_SB = T_SB[:3,:3]

    R_3B = R_S3.T @ R_SB

    theta61 = np.arctan2(R_3B[2,1],-R_3B[2,0])
    theta62 = np.arctan2(-R_3B[2,1],R_3B[2,0])

    theta51 = np.arctan2(np.sqrt(1-R_3B[2,2]**2), R_3B[2,2])
    theta52 = np.arctan2(-np.sqrt(1-R_3B[2,2]**2), R_3B[2,2])

    # theta51 = np.arccos(R_3B[2,2])
    # theta52 = -np.arccos(R_3B[2,2])
    
    theta41 = np.arctan2(R_3B[1,2],R_3B[0,2])
    theta42 = np.arctan2(-R_3B[1,2],-R_3B[0,2])

    thetalist = np.concatenate((i,np.array([theta41,theta51,theta61])))
    thetalist2 = np.concatenate((i,np.array([theta42,theta52,theta62])))
    # print((thetalist*180/np.pi).round(2))
    # print((thetalist2*180/np.pi).round(2))
    # print(mr.FKinSpace(M,Slist,thetalist))
    # print(mr.FKinSpace(M,Slist,thetalist2))
    # print(np.isclose(T_SB,mr.FKinSpace(M,Slist,thetalist),0.01,0.01))
    # print(np.isclose(T_SB,mr.FKinSpace(M,Slist,thetalist2),0.01,0.01))


In [1170]:
mr.FKinSpace(M,Slist,thetalist)

array([[-0.8080127 , -0.53349365, -0.25      , -0.02      ],
       [-0.56959003,  0.8158233 ,  0.09999772,  0.45299982],
       [ 0.15060768,  0.22319693, -0.96306825,  0.81295454],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [1171]:
mr.FKinSpace(M,Slist,thetalist2)

array([[-0.8080127 , -0.53349365, -0.25      , -0.02      ],
       [-0.56959003,  0.8158233 ,  0.09999772,  0.45299982],
       [ 0.15060768,  0.22319693, -0.96306825,  0.81295454],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [1172]:
T_SB

array([[ 0.8080127 , -0.53349365,  0.25      ,  0.02      ],
       [-0.4330127 , -0.25      ,  0.8660254 ,  0.51428203],
       [-0.39951905, -0.8080127 , -0.4330127 ,  0.85535898],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])